In [1]:
import pandas as pd
import numpy as np


In [16]:
# Import data
df = pd.read_csv('430889rows_30052024_with_outliers.csv').fillna('0')
id = pd.read_csv('id_367188rows.csv')
df = df.loc[df['tconst'].isin(id['tconst'])]


In [17]:
# sort by numVote and startYear
df1 = df.sort_values(['numVotes', 'startYear'], ascending=False)
df1 = df1.drop(columns=['producer', 'self', 'writer'], axis=1)

In [18]:
df1.head(3)

,tconst,averageRating,numVotes,titleType,primaryTitle,isAdult,startYear,runtimeMinutes,genres,actor,actress,director
63717,tt0111161,9.3,2888676,movie,The Shawshank Redemption,0.0,1994.0,142.0,Drama,"Tim Robbins,Morgan Freeman,Bob Gunton,William ...",0,Frank Darabont
158940,tt0468569,9.0,2870680,movie,The Dark Knight,0.0,2008.0,152.0,"Action,Crime,Drama","Christian Bale,Christian Bale,Heath Ledger,Aar...","Maggie Gyllenhaal,Monique Gabriela Curnen",Christopher Nolan
225717,tt1375666,8.8,2550842,movie,Inception,0.0,2010.0,148.0,"Action,Adventure,Sci-Fi","Leonardo DiCaprio,Joseph Gordon-Levitt,Elliot ...",Marion Cotillard,Christopher Nolan


In [19]:
# Convert to list of strings
df1[['genres','actor','director', 'actress']] = df1[['genres','actor','director', 'actress']].apply(lambda x: x.str.split(','))

#Create a function in order to remove duplicates of each list 
def remove_dup(lst):
    s = []
    for i in lst:
       if i not in s:
          s.append(i)
    return s

# Remove all duplicats of each list strings in esch column
df1['actor'] = df1['actor'].apply(lambda x: remove_dup(x)) 
df1['director'] = df1['director'].apply(lambda x: remove_dup(x)) 
df1['actress'] = df1['actress'].apply(lambda x: remove_dup(x)) 

# Take 1st value of each column
df1['actor'] = df1['actor'].apply(lambda x: x[0])
df1['actress'] = df1['actress'].apply(lambda x: x[0])
df1['director'] = df1['director'].apply(lambda x: x[0])

In [21]:
# Create numeric label for each category of genres
genres_cat = ['Action', 'Adventure', 'Animation', 'Biography','Comedy', 
               'Crime', 'Documentary', 'Drama', 'Family', 'Fantasy', 
               'Horror','Mystery', 'Romance', 'Sci-Fi', 'Thriller',]

for i in genres_cat:
    df1[i] = df1['genres'].apply(lambda x:1 if i in x else 0)

In [22]:
# Create numeric label for each category of titleType
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
df1['type'] = label_encoder.fit_transform(df1['titleType'])


In [23]:
# Filter top 100 best actor
actor = pd.DataFrame(df1, columns=['tconst', 'numVotes', 'averageRating', 'actor'])
actor_gb = actor.groupby('actor').agg({
                                        'tconst': 'count',
                                        'numVotes': 'sum',
                                        'averageRating': 'mean'
                                                       }).reset_index()
actor_gb_sort = actor_gb.sort_values(['numVotes'], ascending=False).head(10)
lst_actor = actor_gb_sort.actor.tolist()


In [24]:
# Filter top 100 best actress
actress = pd.DataFrame(df1, columns=['tconst', 'numVotes', 'averageRating', 'actress'])
actress_gb = actress.groupby('actress').agg({
                                        'tconst': 'count',
                                        'numVotes': 'sum',
                                        'averageRating': 'mean'
                                                       }).reset_index()
actress_gb_sort = actress_gb.sort_values(['numVotes'], ascending=False).head(10)
lst_actress = actress_gb_sort.actress.tolist()


In [25]:
# Filter top 100 best director
director = pd.DataFrame(df1, columns=['tconst', 'numVotes', 'averageRating', 'director'])
director_gb= director.groupby('director').agg({
                                        'tconst': 'count',
                                        'numVotes': 'sum',
                                        'averageRating': 'mean'
                                                       }).reset_index()
director_gb_sort = director_gb.sort_values(['numVotes'], ascending=False).head(10)
lst_director = director_gb_sort .director.tolist()


In [26]:
# Create numeric label for each name of actors
actor_cat = ['Leonardo DiCaprio', 'Tom Hanks', 'Christian Bale', 'Johnny Depp', 'Tom Cruise', 
              'Robert Downey Jr.', 'Brad Pitt', 'Keanu Reeves', 'Bruce Willis', 'Robert De Niro']

for i in actor_cat:
    df1[i] = df1['actor'].apply(lambda x:1 if i in x else 0)

In [27]:
# Create numeric label for each name of actress
actress_cat = ['Scarlett Johansson', 'Natalie Portman', 'Jennifer Lawrence', 'Cate Blanchett', 'Emma Watson',
              'Anne Hathaway', 'Gwyneth Paltrow', 'Zoe Saldana', 'Keira Knightley', 'Amy Adams']

for i in actress_cat:
    df1[i] = df1['actress'].apply(lambda x:1 if i in x else 0)

In [28]:
# Create numeric label for each name of directors
director_cat = ['Christopher Nolan', 'Steven Spielberg', 'Martin Scorsese', 'Quentin Tarantino', 'David Fincher',
              'Ridley Scott', 'Robert Zemeckis', 'James Cameron', 'Tim Burton', 'Peter Jackson']

for i in director_cat:
    df1[i] = df1['director'].apply(lambda x:1 if i in x else 0)

In [31]:
# Drop non-selected columns
df1 = df1.drop(columns=['genres', 'titleType', 'tconst',
                       'actor', 'actress', 'director'], axis = 1)
# Lower the title's names
df1['primaryTitle']= df1['primaryTitle'].apply(lambda x: x.lower())

In [32]:
df1

,averageRating,numVotes,primaryTitle,isAdult,startYear,runtimeMinutes,Action,Adventure,Animation,Biography,...,Christopher Nolan,Steven Spielberg,Martin Scorsese,Quentin Tarantino,David Fincher,Ridley Scott,Robert Zemeckis,James Cameron,Tim Burton,Peter Jackson
63717,9.3,2888676,the shawshank redemption,0.0,1994.0,142.0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
158940,9.0,2870680,the dark knight,0.0,2008.0,152.0,1,0,0,0,...,1,0,0,0,0,0,0,0,0,0
225717,8.8,2550842,inception,0.0,2010.0,148.0,1,1,0,0,...,1,0,0,0,0,0,0,0,0,0
73838,8.8,2323876,fight club,0.0,1999.0,139.0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
62947,8.8,2256977,forrest gump,0.0,1994.0,142.0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
414188,5.8,5,ponrepovo kouzelnictvi,0.0,1911.0,12.0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
420680,5.2,5,zivot zabité záby,0.0,1911.0,5.0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
420687,5.8,5,chov husí v libusi u prahy,0.0,1911.0,4.0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
247083,5.4,5,rice festival in kyoto,0.0,1910.0,12.0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [1]:
import pandas as pd
df = pd.read_csv('matrix_with_30staffs.csv')

In [2]:
df

,averageRating,numVotes,primaryTitle,isAdult,startYear,runtimeMinutes,Action,Adventure,Animation,Biography,...,Christopher Nolan,Steven Spielberg,Martin Scorsese,Quentin Tarantino,David Fincher,Ridley Scott,Robert Zemeckis,James Cameron,Tim Burton,Peter Jackson
0,9.3,2888676,the shawshank redemption,0.0,1994.0,142.0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,9.0,2870680,the dark knight,0.0,2008.0,152.0,1,0,0,0,...,1,0,0,0,0,0,0,0,0,0
2,8.8,2550842,inception,0.0,2010.0,148.0,1,1,0,0,...,1,0,0,0,0,0,0,0,0,0
3,8.8,2323876,fight club,0.0,1999.0,139.0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
4,8.8,2256977,forrest gump,0.0,1994.0,142.0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
367183,5.8,5,ponrepovo kouzelnictvi,0.0,1911.0,12.0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
367184,5.2,5,zivot zabité záby,0.0,1911.0,5.0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
367185,5.8,5,chov husí v libusi u prahy,0.0,1911.0,4.0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
367186,5.4,5,rice festival in kyoto,0.0,1910.0,12.0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


#

In [2]:
# Set name as index
df = df.set_index('primaryTitle')

from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import NearestNeighbors

# Initialize variable X
X = df.select_dtypes("number")

# Standardize features of X
scaler = StandardScaler()
scaler.fit(X)

X_scaled = pd.DataFrame(scaler.transform(X), index=X.index, columns=X.columns)
X_scaled

,averageRating,numVotes,isAdult,startYear,runtimeMinutes,Action,Adventure,Animation,Biography,Comedy,...,Christopher Nolan,Steven Spielberg,Martin Scorsese,Quentin Tarantino,David Fincher,Ridley Scott,Robert Zemeckis,James Cameron,Tim Burton,Peter Jackson
primaryTitle,,,,,,,,,,,,,,,,,,,,,
the shawshank redemption,2.368364,87.257759,-0.08799,-0.116131,0.737892,-0.276108,-0.223438,-0.259927,-0.186286,-0.569975,...,-0.006392,-0.010438,-0.011314,-0.005717,-0.006175,-0.009039,-0.007563,-0.00595,-0.007741,-0.007194
the dark knight,2.080881,86.713607,-0.08799,0.420412,0.838517,3.621766,-0.223438,-0.259927,-0.186286,-0.569975,...,156.455105,-0.010438,-0.011314,-0.005717,-0.006175,-0.009039,-0.007563,-0.00595,-0.007741,-0.007194
inception,1.889225,77.042533,-0.08799,0.497062,0.798267,3.621766,4.475516,-0.259927,-0.186286,-0.569975,...,156.455105,-0.010438,-0.011314,-0.005717,-0.006175,-0.009039,-0.007563,-0.00595,-0.007741,-0.007194
fight club,1.889225,70.179669,-0.08799,0.075492,0.707704,-0.276108,-0.223438,-0.259927,-0.186286,-0.569975,...,-0.006392,-0.010438,-0.011314,-0.005717,161.946640,-0.009039,-0.007563,-0.00595,-0.007741,-0.007194
forrest gump,1.889225,68.156816,-0.08799,-0.116131,0.737892,-0.276108,-0.223438,-0.259927,-0.186286,-0.569975,...,-0.006392,-0.010438,-0.011314,-0.005717,-0.006175,-0.009039,132.227618,-0.00595,-0.007741,-0.007194
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ponrepovo kouzelnictvi,-0.985609,-0.088179,-0.08799,-3.297065,-0.570237,-0.276108,-0.223438,-0.259927,-0.186286,-0.569975,...,-0.006392,-0.010438,-0.011314,-0.005717,-0.006175,-0.009039,-0.007563,-0.00595,-0.007741,-0.007194
zivot zabité záby,-1.560575,-0.088179,-0.08799,-3.297065,-0.640674,-0.276108,-0.223438,-0.259927,-0.186286,-0.569975,...,-0.006392,-0.010438,-0.011314,-0.005717,-0.006175,-0.009039,-0.007563,-0.00595,-0.007741,-0.007194
chov husí v libusi u prahy,-0.985609,-0.088179,-0.08799,-3.297065,-0.650737,-0.276108,-0.223438,-0.259927,-0.186286,-0.569975,...,-0.006392,-0.010438,-0.011314,-0.005717,-0.006175,-0.009039,-0.007563,-0.00595,-0.007741,-0.007194


In [17]:
#Create a function to return recommendated films

def get_recommendation(title):
    
    # Lower title's name
    title = title.lower()
    
    # To identify the searching film
    # Scale searching value
    input_title_scaled = X_scaled.loc[title].to_frame().T
    
    # Show the KPI of serching film to compare with returned values
    input_title = X.loc[title].to_frame().T
    


    # Fit model
    modelKNN = NearestNeighbors(n_neighbors=2, radius=0.1)
    modelKNN.fit(X_scaled)

    # Identify distance and indeices of similar films
    neigh_dist, neigh_film = modelKNN.kneighbors(input_title_scaled, n_neighbors=10)
    film_similar = neigh_film[0][1:]
    print(df.loc[title])      
    return   X.iloc[film_similar]

In [ ]:
neigh.kneighbors_graph(X)

In [18]:
# Get the user input
title = input("Enter a movie title: ")

get_recommendation(title)

Enter a movie title: top gun
averageRating              6.9
numVotes              502602.0
isAdult                    0.0
startYear               1986.0
runtimeMinutes           109.0
Action                     1.0
Adventure                  0.0
Animation                  0.0
Biography                  0.0
Comedy                     0.0
Crime                      0.0
Documentary                0.0
Drama                      1.0
Family                     0.0
Fantasy                    0.0
Horror                     0.0
Mystery                    0.0
Romance                    0.0
Sci-Fi                     0.0
Thriller                   0.0
type                       0.0
Leonardo DiCaprio          0.0
Tom Hanks                  0.0
Christian Bale             0.0
Johnny Depp                0.0
Tom Cruise                 1.0
Robert Downey Jr.          0.0
Brad Pitt                  0.0
Keanu Reeves               0.0
Bruce Willis               0.0
Robert De Niro             0.0
Scarlett J

,averageRating,numVotes,isAdult,startYear,runtimeMinutes,Action,Adventure,Animation,Biography,Comedy,...,Christopher Nolan,Steven Spielberg,Martin Scorsese,Quentin Tarantino,David Fincher,Ridley Scott,Robert Zemeckis,James Cameron,Tim Burton,Peter Jackson
primaryTitle,,,,,,,,,,,,,,,,,,,,,
the last samurai,7.8,471113,0.0,2003.0,154.0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
collateral,7.5,433146,0.0,2004.0,120.0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
top gun: maverick,8.2,696770,0.0,2022.0,130.0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
magnolia,8.0,328496,0.0,1999.0,188.0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
mission: impossible,7.2,470304,0.0,1996.0,110.0,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
mission: impossible - ghost protocol,7.4,528435,0.0,2011.0,132.0,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
mission: impossible - rogue nation,7.4,410943,0.0,2015.0,131.0,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
mission: impossible iii,6.9,390740,0.0,2006.0,126.0,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
mission: impossible ii,6.1,377703,0.0,2000.0,123.0,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
